In [112]:
# Package import
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

In [113]:
# Data import and reading/assignation of values

file_path = Path('DataRefValue.xlsx')

data = pd.read_excel(file_path, sheet_name= 'Sheet1')

ref_values = data['RefValue']

value_columns = ['Value1', 'Value2', 'Value3', 'Value4', 'Value5', 'Value6'] 

#data = data.dropna(subset=value_columns, how='all')

#ref_values = data['RefValue']
time_periods = ['Duration1', 'Duration2', 'Duration3', 'Duration4', 'Duration5', 'Duration6']
#Evalue_columns = ['Value1', 'Value2', 'Value3', 'Value4', 'Value5', 'Value6'] 

valid_ref_values = data.loc[~data['Value1'].isna(), 'RefValue']

value_types = ['Emergency Response', 'Occupational', 'General Public']

data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

data.head(10)

,RefValue,Value1,Duration1,Value2,Duration2,Value3,Duration3,Value4,Duration4,Value5,Duration5,Value6,Duration6,Type,Shape,MarkColor,LineDash,Outline,CatColor
0,AEGL-1,420.0,0.17,240.0,0.5,170.0,1.0,58.0,4.0,29.0,8.0,NaN,NaN,Emergency Response,Diamond,Blue,Solid,Yes,Red
1,AEGL-2,6500.0,0.17,3600.0,0.5,2600.0,1.0,1300.0,4.0,650.0,8.0,NaN,NaN,Emergency Response,Diamond,Gold,Solid,Yes,Red
2,AEGL-3,31000.0,0.17,18000.0,0.5,13000.0,1.0,6500.0,4.0,3300.0,8.0,NaN,NaN,Emergency Response,Diamond,Red,Solid,Yes,Red
3,ERPG-1,156.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Triangle,Red,NaN,Yes,Red
4,ERPG-2,470.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Triangle,Gold,NaN,Yes,Red
5,ERPG-3,3190.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Triangle,Blue,NaN,Yes,Red
6,PAC-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Diamond,Red,NaN,Yes,Red
7,PAC-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Diamond,Gold,NaN,Yes,Red
8,PAC-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Diamond,Blue,NaN,Yes,Red
9,ACGIH-Ceiling*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Occupational,Circle,Red,NaN,Yes,Orange


In [114]:
fig = go.Figure()

'''
# Colors for legend categories, added as CatColors
category_colors = {
    "Emergency Response" : 'Red',
    "Occupational" : 'Gold',
    "General Public" : 'Green',
}
'''
shape_mapping = {
    "Diamond": "diamond",
    "Triangle": "triangle-up",
    "Circle": "circle",
    "Square": "square",
    "Cross": "cross",
    "Circle-Open": "circle-open",
}

color_mapping = {
    "Red": "red",
    "Gold": "gold",
    "Blue": "#0085AA",
    "Orange": "orange",
    "Yellow": "yellow",
    "Peach": "#FFDAB9",
    "Beige": "#F5F5DC",
    "White": "white",
    "Pink": "pink",
    "Light Green": "#90EE90",
    "Green": "#008700",
    "Pale Green": "#98FB98",
}

line_mapping = {
    "Solid": "solid",
    "Dash": "dash",
    "Dot": "dot",
    "Dashdot": "dashdot",
    "None" : "none",
}

outline_mapping = {
    "Yes": 1,
    "No": 0,
}


for idx, ref_value in enumerate(ref_values):
    durations = data.loc[idx, time_periods].dropna().values
    values = data.loc[idx, value_columns].dropna().values
    
    shape = data.loc[idx, 'Shape']
    color_name = data.loc[idx, 'MarkColor']
    outline = data.loc[idx, 'Outline']
    line_type = data.loc[idx, "LineDash"]

    marker_symbol = shape_mapping.get(shape, "circle")
    marker_color = color_mapping.get(color_name, "gray")
    marker_outline = outline_mapping.get(outline, "black")

    line_appearance = line_mapping.get(line_type, "solid")

    showlegend = ref_value in valid_ref_values


    category = data.loc[idx, 'Type']

    if category == "Emergency Response":
        legend_title = "Emergency Response"
        legend_bgcolor = "Red"
    elif category == "Occupational":
        legend_title = "Occupational"
        legend_bgcolor = "Gold"
    else:
        legend_title = "General Public"
        legend_bgcolor = "Green"


    fig.add_trace(go.Scatter(
        x=durations,
        y=values,
        mode='lines+markers', # +text',
        name=ref_value,
        hoverlabel = dict(bgcolor="white", font_size = 16, font_family = "Rockwell", font_color = "black"),

        hovertemplate=(
            "<b>%{customdata}</b><br>"
            "x: %{x}<br>"
            "y: %{y}"
            "<extra></extra>"
        ),

        customdata=[ref_value]*len(durations), 
        legendgroup=category,

        marker=dict(symbol=marker_symbol, color=marker_color, size = 8, line=dict(width = marker_outline, color = "black")),
        line=dict(dash=line_appearance),
        #text = [ref_value], # * len[values],
        #textposition="top center"
    ))

annotations = []

legend_x = 1.05
legend_y = 1.0
legend_gap = 0.1

legend_y2 = 1.0
legend_gap2 = 0.1


# Set axes and add legend
# TODO: Get legend in 2 columns, avoid scrolling
fig.update_layout(
    title="Benzene Inhalation Reference Values",
    xaxis_title="Duration (Hours)",
    yaxis_title="Concentration mg/m3",
    xaxis_type="log",
    yaxis_type="log",
    template="plotly_white",
    height = 800,
    width = 1000,

    # hovermode="x unified",

    legend = dict(
        # groupclick = "toggleitem",
        tracegroupgap = 5,
        #orientation = "h",
        #x=0,
        #y=-0.2,
        #traceorder="normal",
        # bgcolor = "LightSteelBlue",
        font=dict(
            color = "Grey",
            size=8
        )
    ),
    annotations = annotations

)

# This stuff causes issues when placed before update layout chunk

# lines to add, specified by x-position
lines = {'24 Hours':24,'30 Days':720,'7 years':61320,'70 Years':613200,'Cancer Ranges':675000}



# Actually adds the lines
for label, x_position in lines.items():
    #print(k)
    fig.add_shape(type='line',
                yref="paper",
                x0=x_position,
                y0=0,
                x1=x_position,
                y1=1,
                line=dict(color='#0085AA', width=2))
    
    # Create scatter trace of text labels
    fig.add_trace(go.Scatter(
        x=[x_position],
        # yref = 'paper',
        y = [1],
        mode = "markers",
        marker=dict(opacity=0),
        hoverinfo="text",
        hoverlabel = dict(bgcolor="white", font_color="black"),
        text=[label],
        showlegend=False
    ))

fig.add_shape(
    type="rect",
    xref="paper",
    x0=0,
    x1=0.34,
    yref="paper",
    y0=.95,
    y1=1,
    fillcolor="rgba(255,255,102,0.5)",
    line=dict(color="black", width=0.4)
)

fig.add_annotation(
    xref="paper",
    x=.1,
    yref="paper",
    y=.99,
    text="Acute Effects",
    showarrow=False,
    font=dict(size=12,color="black"),
    align="center"
)

fig.add_shape(
    type="rect",
    xref="paper",
    x0=0.34,
    x1=0.535,
    yref="paper",
    y0=.95,
    y1=1,
    fillcolor="rgba(255,255,102,0.5)",
    line=dict(color="black", width=0.4)
)

fig.add_annotation(
    xref="paper",
    x=.44,
    yref="paper",
    y=.99,
    text="Short Term",
    showarrow=False,
    font=dict(size=12,color="black"),
    align="center"
)

fig.add_shape(
    type="rect",
    xref="paper",
    x0=0.535,
    x1=0.785,
    yref="paper",
    y0=.95,
    y1=1,
    fillcolor="rgba(255,255,102,0.5)",
    line=dict(color="black", width=0.4)
)

fig.add_annotation(
    xref="paper",
    x=.7,
    yref="paper",
    y=.99,
    text="Subchronic",
    showarrow=False,
    font=dict(size=12,color="black"),
    align="center"
)

fig.add_shape(
    type="rect",
    xref="paper",
    x0=0.785,
    x1=1,
    yref="paper",
    y0=.95,
    y1=1,
    fillcolor="rgba(255,255,102,0.5)",
    line=dict(color="black", width=0.4)
)

fig.add_annotation(
    xref="paper",
    x=.92,
    yref="paper",
    y=.99,
    text="Chronic",
    showarrow=False,
    font=dict(size=12,color="black"),
    align="center"
)

fig.show()